
### Търсене и извличане на информация. Приложение на дълбоко машинно обучение
### Стоян Михов
### Зимен семестър 2025/2026

## Упражнение 4

За да работи програмата трябва корпуса от публицистични текстове за Югоизточна Европа,
да се намира разархивиран в наддиректорията, в която е програмата (виж упражнение 2).

Преди да се стартира програмата е необходимо да се активира съответното обкръжение с командата:
conda activate tii

In [1]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import sys
import math
corpus_root = '../JOURNALISM.BG/C-MassMedia'
myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')
fileNames = myCorpus.fileids()

<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Rumen\AppData\Local\Temp\ipykernel_22136\3181726213.py:6: SyntaxWarning: invalid escape sequence '\.'
  myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')


In [2]:
class progressBar:
    def __init__(self, barWidth=80):
        self.barWidth = barWidth
        self.period = None

    def start(self, count):
        self.period = int(count / self.barWidth)
        sys.stdout.write("["+(" " * self.barWidth)+"]\n")
        sys.stdout.flush()
        sys.stdout.write("[") # for jupyter
        sys.stdout.flush() # for jupyter
        # sys.stdout.write("\b" * (self.barWidth+1)) # for terminal

    def tick(self, item):
        if (item+1) % self.period == 0:
            sys.stdout.write("#")
            sys.stdout.flush()

    def stop(self):
        sys.stdout.write("]\n")

In [3]:
def printDocuments(myCorpus,fileNames,docList):
    for docID,score in docList:
        text = myCorpus.words(fileNames[docID])
        print('Document ID: '+str(docID)+', relevance: '+str(score))
        for word in text:
            print(word,end=' ')
        print('\n')

Дефинираме $tf_{t,d}$  (term frequency), като броя на срещанията на терма $t$ в документа $d$.
$$
w_{t,d} =
\begin{cases}
1 + \log \mathrm{tf}_{t,d}, & \text{if } \mathrm{tf}_{t,d} > 0 \\
0, & \text{otherwise}
\end{cases}
$$
Дефинираме $df_t$ (document frequency) като броя на документите, в които се среща термът $t$.
Дефинираме обратната документна честота $idf_t$:
$$idf_t = \log \frac{N}{df_t}  $$
Дефинираме теглото $tf\cdot idf$ като:
$$tf \cdot idf_{t,d} = \log (1 + tf_{t,d} ) \times log \frac{N}{df_t}$$

In [4]:
def createIndex(myCorpus, fileNames):
    pb = progressBar()
    pb.start(len(fileNames))
    dictionary={}
    for docID, fileName in enumerate(fileNames):
        pb.tick(docID)
        text = myCorpus.words(fileName)
        for token in text:
            if not token.isalpha(): continue
            term = token.lower()
            if term in dictionary:
                (postings, tf) = dictionary[term]
                if postings[-1] < docID:
                    postings.append(docID)
                    tf.append(1)
                else:
                    tf[-1] += 1
            else:
                dictionary[term] = ([docID],[1])
    pb.stop()
    N = len(fileNames)
    pb.start(len(dictionary))
    lengths = [0.0]*N # дължината на векоторите на документите
    t=0
    for term in dictionary:
        pb.tick(t)
        t += 1
        (postings, tf) = dictionary[term]
        df = len(postings) # брой документи, в които се среща термът
        idf = math.log(N/df) # обратна документна честота
        for i, docID in enumerate(postings):
            w = math.log(1+tf[i]) * idf
            lengths[docID] += w ** 2 
    for docID in range(N):
        lengths[docID] = math.sqrt(lengths[docID])
    pb.stop()
    return dictionary, lengths

In [5]:
dictionary, lengths = createIndex(myCorpus, fileNames)

[                                                                                ]
[################################################################################]
[                                                                                ]
[################################################################################]


$$
\cos(\vec{q}, \vec{d}) = \frac{\vec{q} \cdot \vec{d}}{|\vec{q}|\,|\vec{d}|}
= \frac{\sum_{i=1}^{M} q_i d_i}{\sqrt{\sum_{i=1}^{M} q_i^2} \sqrt{\sum_{i=1}^{M} d_i^2}}
$$


In [6]:
def cosineScore(query, K, dictionary, lengths):
    N = len(lengths)
    scores = {}
    for term in query:
        if term not in dictionary: continue
        (postings, tf) = dictionary[term]
        df_t = len(postings)
        idf_t = math.log(N/df_t)
        #        w_tq = idf_t
        w_tq = 1.0
        for i, docID in enumerate(postings):
            w_td = math.log(1+tf[i]) * idf_t
            if docID not in scores: scores[docID]=0.0
            scores[docID] += w_tq * w_td
    for docID in scores:
        scores[docID] = scores[docID] / lengths[docID]
    result = sorted([(docID,scores[docID]) for docID in scores], key = lambda x: x[1] , reverse=True)
    return result[:K]

In [7]:
printDocuments(myCorpus, fileNames, cosineScore(['световно', 'първенство', 'по', 'футбол'], 5, dictionary, lengths))

Document ID: 31104, relevance: 0.45938644151362595
Словения извоюва историческа победа в мач за Световната купа 14 / 06 / 2010 КЕЙПТАУН , ЮАР -- Словения отбеляза в неделя ( 13 юни ) първата си победа на световно първенство по футбол , като победи Алжир с 1 : 0 . Голът бе дело на капитана на словенския отбор , Роберт Корен , в 79 - ата минута . Другият мач от Група C , между Англия и Съединените щати , завърши с равенство 1 : 1 , което даде на Словения преднина в групата . Словения играе срещу САЩ в петък . ( AФП , АП , Уебсайт на ФИФА - 13 / 06 / 10 ) 

Document ID: 16138, relevance: 0.4032271663021162
ФИФА критикува Албания за изстъпления на фенове 10 / 03 / 2005 ЛОНДОН , Великобритания -- ФИФА , световната федерация по футбол , отсъди в сряда ( 9 март ) Албания да заплати глоба от 40000 швейцарски франка и да играе следващите си два домакински мача от квалификациите за световно първенство пред празни трибуни . Решението бе взето в отговор на грозните сцени по време на мача срещу Укр

In [8]:

printDocuments(myCorpus, fileNames, cosineScore(['румъния','вирус'], 5, dictionary, lengths))

Document ID: 31655, relevance: 0.3676287454776024
Потвърдено е наличието на рядък вирус в Гърция 10 / 08 / 2010 АТИНА , Гърция -- Здравните власти потвърдиха в понеделник ( 9 август ), че 16 души са се заразили със западнонилски вирус , а трима от тях са починали . Много хора , заразени със западнонилския вирус , който обикновено се пренася от комари , изобщо не се разболяват , но някои развиват смъртоносен менингит и енцефалит . ( АП , АНА - МПА - 09 / 08 / 10 ) 

Document ID: 31756, relevance: 0.36378084454958454
Западнонилският вирус причини смъртта на още трима души в Гърция 20 / 08 / 2010 АТИНА , Гърция -- Общо седем са вече смъртните случаи от западнонилския вирус , след като още трима възрастни хора починаха от възпаление на мозъка , съобщиха здравните власти в четвъртък ( 19 август ). За други 77 души в Централна и Северна Гърция е установено , че са заразени с вируса . Западнонилският вирус се разпространява от комари . ( AП , ИТАР - ТАСС - 19 / 08 / 10 ) 

Document ID: 31878,